### GIS Process

In [4]:
# Libraries

import rasterio
from rasterio.features import shapes
from rasterio.mask import mask as mask
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.features import shapes
from rasterio.plot import show 
from shapely.geometry import shape
import numpy as np
import fiona
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from osgeo import gdal
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.patches as mpatches
import seaborn as sns

print(f'Rasterio version: {rasterio.__version__}')
print(f'fiona version: {fiona.__version__}')

Rasterio version: 1.2.6
fiona version: 1.8.20


In [3]:
# Defining variables and paths:

YEAR = 2020
ROI = 'Area'

ROI_SHAPE = "data/file.shp"
ROI_LUC = f"data/file.tif"
ROI_SCAR = f"data/file.tif"


export_tif_path_luc = f"data/MOSAICO_{YEAR}_LUC_{ROI}.tif"
export_shp_path_luc = f"data/MOSAICO_{YEAR}_LUC_{ROI}.shp"
export_tif_path_scar = f"data/MOSAICO_{YEAR}_{ROI}.tif"

In [5]:
def cropa_tif_from_shp(shp_path, tif_path, export_tif_path):
    print(f'- Cropando {tif_path} com {shp_path}')
    with fiona.open(shp_path, "r") as shapefile:
        roi_shapes = [feature["geometry"] for feature in shapefile]
    
    tif_open = rasterio.open(tif_path)
    masked_tif, masked_tif_transform = rasterio.mask.mask(tif_open, roi_shapes, crop=True) 
    scar_meta = tif_open.meta
    
    scar_meta.update({ 
                 "height": masked_tif.shape[1],
                 "width": masked_tif.shape[2],
                 "transform": masked_tif_transform,
                 "nodata": 0})
    
    tif_result = np.where(masked_tif==256, 0, masked_tif)
    
    print(f'Salvando em {export_tif_path}\n')
    with rasterio.open(export_tif_path, "w", **scar_meta) as dest:
        dest.write(tif_result)

In [ ]:
# Clip LUC por ROI
cropa_tif_from_shp(ROI_SHAPE, ROI_LUC, export_tif_path_luc)

In [ ]:
# LUC and ROI
luc_open = rasterio.open(export_tif_path_luc)

# LUC variable
luc_roi = luc_open.read(1)

In [ ]:
# Visualizing land use and cover raster file (Region of Interest):

show(luc_open, transform=luc_open.transform, cmap='viridis')

In [6]:
# Defining poligonize function

def poligoniza(tiff_path, shp_path):
    cmd_command = f'gdal_polygonize.py {tiff_path} {shp_path}'
    print(f'- Poligonizando {tiff_path} -> {shp_path}')
    os.system(cmd_command)
    print('')

In [7]:
# Defining clip and poligonize function

def cropa_e_poligoniza(input_shp_path, input_tif_path, output_shp_path):
    crop_tif_path = output_shp_path.replace('.shp', '.tif')
    cropa_tif_from_shp(input_shp_path, input_tif_path, crop_tif_path)
    poligoniza(crop_tif_path, output_shp_path)

In [ ]:
# Cliping and poligonizing function

cropa_e_poligoniza(ROI_SHAPE, ROI_LUC, export_shp_path_luc)

In [ ]:
# Conversion TIF to SHP
def tif_to_shp (tif_path, shape_path):
    print (f'Convertendo {tif_path} para {shape_path}')
    os.system("gdal_polygonize.py data/MOSAICO_..._.tif data/MOSAICO_2020_....shp -f 'ESRI Shapefile'")

In [ ]:
tif_to_shp(export_tif_path_luc, export_shp_path_luc)